In [3]:
#Code developed by Andrew Steiner, Mineral Deposit Research Unit, University of British Columbia, 2023
#USER NEEDS TO SET THE FOLDER CONTAINING ALL OF THE .TXT FILES (NOTE: IT MUST BE IN THE SAME FOLDER AS THIS NOTEBOOK) IN LINE 4 AND THE OUTPUT FILE NAME IN THE LINE 84

folder = "TXTCompile_TEMP"



def xrf_importer(folder):
    # Import required packages
    import glob
    import os
    import pandas as pd

    # Saves the first micro XRF text file in an object called 'xrf_file'
    folder = folder.rstrip('/')
    xrf_file = folder + """/""" + os.listdir(folder)[0]

    # Retrieve all files ending in .txt
    all_files = glob.glob(os.path.join(folder, "*.txt"))

    # Creates empty list
    database_list = []

    for filename in all_files:
        # Reads in each text file, converts it to a dataframe, duplicates the index column and names it the y coordinate
        df = pd.read_csv(filename, delimiter=";", header=None)
        # df = pd.read_csv(filename, delimiter=",", header='wavelength')
        df = df.reset_index().rename(columns={"index": "y"})
        # Names the column as the element from the text file name
        # Strips '.txt' from end of filename
        filename = filename[:-4]
        # Removes '-K', '-Ka' or '-La' from end of name
        
        
        suffixes = ['-K', '-Ka', '-La', '-Kα', '-Lα', '-ka', '-la', '-k']
        for suffix in suffixes:
            if filename.endswith(suffix):
                filename = filename[:-len(suffix)]
                break
        
        
        # Assigns column name to last two remaining digits and strips the '_' if present
        column_name = filename[-2:].replace("_", "")
        # Melts the dataframe into x, y and element column with each row representing a pixel
        df = (df.melt(id_vars="y",
                      var_name="x",
                      value_name=column_name)
              )
        # Appends melted dataframe to database_list
        database_list.append(df)

    # Concatenates each element's dataframe into a single dataframe
    xrf_df = pd.concat(database_list, axis=1)
    # Removes duplicated columns
    xrf_df = xrf_df.loc[:, ~xrf_df.columns.duplicated()]

    if 'eo' in xrf_df.columns:
        # Corrects name for video column
        xrf_df = xrf_df.rename(columns={"eo": "Video"})
        # Moves video column to start of dataframe
        vid = xrf_df["Video"] # Saves video column as object called 'vid'
        xrf_df = xrf_df.drop(columns="Video") # Drops video column
        xrf_df.insert(0, "Video", vid) # Inserts vid object as first column

    # Cast x and y columns as integers
    xrf_df["x"] = xrf_df["x"].astype(int)
    xrf_df["y"] = xrf_df["y"].astype(int)

    # Stores the width of the map as objects
    width = round(len(pd.read_csv(xrf_file, delimiter=';', header=None).columns))

    # Sets coordinate (1,1) to bottom left of data so that it plots correctly
    xrf_df['y'] = 1 - xrf_df['y']
    xrf_df['x'] = width + xrf_df['x']

    # Set x and y as multi-index
    xrf_df.set_index(["x", "y"], inplace=True)
    xrf_df.sort_index(inplace=True)

    return xrf_df

df = xrf_importer(folder)

df.to_csv('19FB-131_MassPctNorm.csv')